In [1]:
import os
import pickle
import time
import collections
import sys

import cv2
import argparse
import numpy as np
from IPython import display

import zipfile
import io
import inspect

from openvino.inference_engine import IECore

In [2]:
# Load the model
model_xml = 'image-retrieval-0001/FP16/image-retrieval-0001.xml'
model_bin = 'image-retrieval-0001/FP16/image-retrieval-0001.bin'

In [3]:
model_name = 'image-retrieval-0001'
base_model_dir = 'image-retrieval-0001'
precision = 'FP16'

In [4]:
if not os.path.exists(model_xml):
    convert_command = f"omz_converter " \
                      f"--name {model_name} " \
                      f"--download_dir {base_model_dir} " \
                      f"--precisions {precision}"
    ! $convert_command

In [5]:
ie = IECore()
net = ie.read_network(model=model_xml, weights=model_bin)
# net = net.reshape([1, 3, 224, 224]) # here I want to change input shape of model, but it ask's for a dict instead list
exec_net = ie.load_network(network=net, device_name='CPU')

In [13]:
net.input_info

{'Placeholder': <openvino.inference_engine.ie_api.InputInfoPtr at 0x27eb0cbd0e0>}

In [6]:
input_blob = next(iter(net.input_info))
out_blob = next(iter(net.outputs))

In [7]:
def preprocess_image(image_path, h, w):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (w, h))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype(np.float32)  # Convert to float32
#     image = image.transpose((2, 0, 1))  # Interchange dimensions to [1, 3, 224, 224]
    image = np.array(image)
    blob = np.expand_dims(image, axis=0)  # Add batch dimension
    print(blob.shape)
    return blob

In [16]:
# Preprocess the batch of images
batch_images_path = 'archive/batchZip.zip'
batch_images_dir = 'unpack_archive'


with zipfile.ZipFile(batch_images_path, 'r') as zip_ref:
    zip_ref.extractall(batch_images_dir)
    zip_ref.close()

batch_images = []
for image_file in os.listdir(batch_images_dir):
    image_path = os.path.join(batch_images_dir, image_file)
    preprocessed_image = preprocess_image(image_path, 224, 224)
    batch_images.append(exec_net.infer(inputs = {input_blob: preprocessed_image}))
# написать цикл сравнения batch_images и главной картинки (output)!!!!
# batch_images = np.array(batch_images)

(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)


In [20]:
net.input_info


{'Placeholder': <openvino.inference_engine.ie_api.InputInfoPtr at 0x27eb0cbf420>}

In [24]:
# Данный блок кода из лекции, его необходимо распределить по приложению!!!!! 
input_blob = next(iter(net.input_info))
out_blob = next(iter(net.outputs))
n, h, w, c = net.input_info[input_blob].input_data.shape
# Load, transpose, expand operations
blob = preprocess_image('images/knf.jpg', w, h)
# Execute
output = exec_net.infer(inputs = {input_blob: blob})
output = output[out_blob]

(1, 224, 224, 3)


In [25]:
output

array([[ 6.22731298e-02,  4.34976146e-02,  9.02910456e-02,
        -1.50097429e-03, -1.35600092e-02,  1.73886511e-02,
        -4.19101417e-02,  9.76884086e-03,  1.98603626e-02,
        -8.59015584e-02, -3.74260452e-03,  5.12176231e-02,
         9.49959606e-02, -1.00796126e-01,  1.56500600e-02,
        -7.90645108e-02,  2.16081440e-02, -4.41139489e-02,
         1.13691247e-04,  1.42992780e-01, -2.07431260e-02,
         9.93202627e-02, -4.24874425e-02, -5.36948331e-02,
        -2.41894964e-02,  6.92646503e-02, -2.42105369e-02,
         9.84907802e-03,  1.59111582e-02, -6.95780590e-02,
        -2.67326552e-02, -3.44167873e-02,  2.67278720e-02,
         5.29089868e-02, -1.99196438e-05, -6.36052266e-02,
        -1.50818661e-01,  2.26238891e-02,  8.76223966e-02,
         2.72878613e-02, -4.55431864e-02, -1.97101049e-02,
         1.81965083e-02, -6.81264549e-02, -9.80198309e-02,
         4.06974182e-03, -1.43808732e-02, -4.49153706e-02,
         4.72197607e-02, -9.22180414e-02,  4.03433144e-0

Дальше попробуем загрузить изображение в модель последовательно руками

In [ ]:
print('N = ', N, "|",'C = ', C, "|", 'H = ', H, "|", 'W = ', W, "|")

In [ ]:
input_image.shape

In [ ]:
output = exec_net.infer(inputs = {input_blob: input_image})

In [ ]:
image = preprocess_image("images/knf.jpg")

In [ ]:
net.input_info[input_blob].input_data.shape

In [ ]:
blob.shape

In [ ]:
# initialize inference engine
ie_core = IECore()
# read the network and corresponding weights from file
net = ie_core.read_network(model=model_xml)
# load the model on the CPU (you can choose manually CPU, GPU, MYRIAD etc.)
# or let the engine choose best available device (AUTO)
exec_net = ie_core.load_network(network=net, device_name="CPU")

# get input and output names of nodes
input_key = list(exec_net.input_info)[0]
output_key = list(exec_net.outputs.keys())[0]

# get input size
height, width = exec_net.input_info[input_key].tensor_desc.dims[2:]

In [ ]:
output_key

# тут roi_matching.py из проекта imgcls

In [ ]:
class ImageClassifier:
    def __init__(self):
        self.all_skus = {}
        self.model = model_util.DeepModel()
        self.predict_time = 0
        self.time_search = 0
        self.count_frame = 0
        self.top_k = 5

    def extract_features_from_img(self, cur_img):
        """Судя по названию эта функция извлекает фичи из тестового изображения"""
        cur_img = cv2.resize(cur_img, (224, 224))
        img = preprocess_input(cur_img)
        img = np.expand_dims(img, axis=0)
        feature = self.model.extract_feature(img)
        return feature

    def predict(self, img):
        self.count_frame += 1
        before_time = time.time()
        target_features = self.extract_features_from_img(img)
        self.predict_time += time.time() - before_time
        max_distance = 0
        result_dish = 0

        for dish, features_all in self.all_skus.items():
            for features in features_all:
                cur_distance = self.model.cosine_distance(target_features, features)
                cur_distance = cur_distance[0][0]
                if cur_distance > max_distance:
                    max_distance = cur_distance
                    result_dish = dish

        return result_dish, max_distance

    def add_img(self, img_path, id_img):
        img = cv2.imread(img_path)
        cur_img = img
        feature = self.extract_features_from_img(cur_img)
        if id_img not in self.all_skus:
            self.all_skus[id_img] = []
        self.all_skus[id_img].append(feature)
        return feature

    def remove_by_id(self, id_img):
        if id_img in self.all_skus:
            self.all_skus.pop(id_img)

    def remove_all(self):
        self.all_skus.clear()

    def add_img_from_pickle(self, id_img, pickle_path):
        res = pickle.load(open(pickle_path, 'rb'))
        self.all_skus[id_img] = res

    def get_additional_info(self):
        json_res = {}
        json_res["Extract features, time"] = self.predict_time
        json_res["Find nearest, time"] = self.time_search
        json_res["Count frame"] = self.count_frame
        json_res["RPS"] = self.count_frame / (self.predict_time + self.time_search)
        return json_res